In [ ]:
from pymongo import MongoClient

In [ ]:
# Instantiate database
db = MongoClient().rep_db

In [1]:
import configparser

In [2]:
# Get Propublica API KEY
config = configparser.ConfigParser()
config.read('./config.ini')
PROPUBLICA_API_KEY = config.get('propublica', 'PROPUBLICA_API_KEY')

In [13]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [4]:
API_ROOT = 'https://api.propublica.org/congress/v1/'
header = {'X-API-Key': f'{PROPUBLICA_API_KEY}'}

In [70]:
def get_house_ids(congress):
    '''
    Function to get house members by congress number
    '''

    call_string = API_ROOT + f'{congress}/house/members.json'
    r = requests.get(call_string, headers=header)
    result = r.json()['results'][0]['members']
    member_ids = [ member['id'] for member in result ]
    
    return member_ids

In [92]:
def get_member_data(member):
    call_string = API_ROOT + f'members/{member}.json'
    r = requests.get(call_string, headers=header)
    result = r.json()['results'][0]
    
    return result

In [93]:
test = get_house_ids(117)

In [94]:
a = get_member_data(test[0])

In [95]:
a

{'id': 'A000370',
 'member_id': 'A000370',
 'first_name': 'Alma',
 'middle_name': '',
 'last_name': 'Adams',
 'suffix': None,
 'date_of_birth': '1946-05-27',
 'gender': 'F',
 'url': 'https://adams.house.gov',
 'times_topics_url': '',
 'times_tag': '',
 'govtrack_id': '412607',
 'cspan_id': '76386',
 'votesmart_id': '5935',
 'icpsr_id': '21545',
 'twitter_account': 'RepAdams',
 'facebook_account': 'CongresswomanAdams',
 'youtube_account': None,
 'crp_id': 'N00035451',
 'google_entity_id': '/m/02b45d',
 'rss_url': 'https://adams.house.gov/rss.xml',
 'in_office': True,
 'current_party': 'D',
 'most_recent_vote': '2021-01-21',
 'last_updated': '2021-01-26 18:30:36 -0500',
 'roles': [{'congress': '117',
   'chamber': 'House',
   'title': 'Representative',
   'short_title': 'Rep.',
   'state': 'NC',
   'party': 'D',
   'leadership_role': None,
   'fec_candidate_id': 'H4NC12100',
   'seniority': '10',
   'district': '12',
   'at_large': False,
   'ocd_id': 'ocd-division/country:us/state:nc/cd

In [56]:
def get_roll_call_vote(congress, session_number, roll_call_num):
    query = f'{congress}/house/sessions/{session_number}/votes/{roll_call_num}.json'
    call_string = API_ROOT + query
    r = requests.get(call_string, headers=header)
    result = r.json()['results']['votes']
    
    return result

In [57]:
test = get_roll_call_vote(116, 1, 1)

In [60]:
test

{'vote': {'congress': 116,
  'session': 1,
  'chamber': 'House',
  'roll_call': 1,
  'source': 'http://clerk.house.gov/evs/2019/roll001.xml',
  'url': 'http://clerk.house.gov/evs/2019/roll001.xml',
  'bill': {'bill_id': 'quorum-116',
   'number': 'QUORUM',
   'api_uri': None,
   'title': None,
   'short_title': None,
   'latest_action': None},
  'amendment': {},
  'question': 'Call by States',
  'question_text': '',
  'description': '',
  'vote_type': 'QUORUM',
  'date': '2019-01-03',
  'time': '12:39:00',
  'result': 'Passed',
  'democratic': {'yes': 0,
   'no': 0,
   'present': 234,
   'not_voting': 1,
   'majority_position': 'Present'},
  'republican': {'yes': 0,
   'no': 0,
   'present': 197,
   'not_voting': 2,
   'majority_position': 'Present'},
  'independent': {'yes': 0, 'no': 0, 'present': 0, 'not_voting': 0},
  'total': {'yes': 0, 'no': 0, 'present': 431, 'not_voting': 3},
  'positions': [{'member_id': 'A000374',
    'name': 'Ralph Abraham',
    'party': 'R',
    'state': 'LA